In [1]:
import pandas as pd
import pyodbc as db
import db_config as config
import numpy as np
import time
import json

#### Connect to the Database

In [2]:
conn = db.connect(DRIVER='SQL Server',
                 SERVER = config.server_name,
                 UID = config.user,
                 PWD=config.pwd,
                 DATABASE=config.database_name)

In [6]:
with open('SysOfferings.json', 'r') as f:
    custs = json.load(f)

In [8]:
custs

{'Boiler': ['Ackumen Solid Feeder',
  'Inventory',
  'Remote Corrosion',
  'Ackumen Boiler Management',
  'RBA-i'],
 'Cooling Water': ['Ackumen Cooling Management',
  'Ackumen Solid Feeder',
  'Inventory',
  'Remote Corrosion'],
 'Fermentation Line': ['Inventory',
  'Ackumen ECHOWISE Pro Visible fermentation'],
 'Pulp Washing': ['Inventory', 'Ackumen ECHOWISE Expert Supervisory Control'],
 'Paper Machine': ['Inventory', 'MCA-i'],
 'Closed Loop System': ['Inventory', 'Ackumen Closed Loop']}

In [3]:
conn

#### Read the Tags from the excel sheet and modify data

In [22]:
monitor = pd.read_csv('RegisterData.csv')
monitor2= monitor.dropna(subset='Address').reset_index().drop(columns='index')
monitor2['Address'] = [int(i) for i in monitor2['Address']]

In [23]:
cols = monitor2.columns.tolist()
for col in cols:
    monitor2[col] = [str(i) for i in monitor2[col]]

In [24]:
monitor2

,Address,Register name,Scale,Description
0,104,PumpPowerON,nan,Power State - 0 or 1
1,207,ActualSetpointManualHI,0.1 ml/h,"The actual setpoint used in operating mode ""Ma..."
2,208,ActualSetpointManualLO,nan,Can be set via SetpointManual (register 00106-...
3,209,ActualPulseVolumeHI,1 nl,The actual pulse volume used in operating mode...
4,210,ActualPulseVolumeLO,nan,Can be set via SetPulseVolume (register 00108-...
5,211,ActualBatchDosingVolumeHI,0.001 ml,Can be set via SetBatchDosingVolume (register ...
6,212,ActualBatchDosingVolumeLO,nan,HMI.
7,213,ActualBatchDosingTimeHI,0.1 s,The actual batch dosing time used in operating...
8,214,ActualBatchDosingTimeLO,nan,Can be set via SetBatchDosingTime (register 00...
9,215,ActualPressureMax,0.1 bar,Actual value of (relative) pressure alarm limi...


In [25]:
monitor2 = monitor2.replace('nan', 'NA')

In [26]:
cols

['Address', 'Register name', 'Scale', 'Description']

#### Custom functions to create table, Write, read and delete tags

In [27]:
def createTable(db_name, tablename, columns, conn):
    insertCMD = f'''CREATE TABLE "{tablename}" ({columns});'''
    cur = conn.cursor()
    try:
        cur.execute(insertCMD)
        conn.commit()
        print(f"Table {tablename} was created in DB {db_name}")
    except(Exception, db.DatabaseError) as error:
        print(error)

In [28]:
def writeValues(metrics, conn, table):
    try:
        cur = conn.cursor()
    except (Exception, db.DatabaseError) as error:
        print(error)
    keys = list(metrics.keys())
    values = tuple(metrics.values())
    cols = '"' + ('","').join(keys) + '"'
    s_lens = "?,"*len(keys)
    s = s_lens.split(",")
    s = (",").join(s[:-1])
    insertQ = f""" INSERT INTO {table} ({cols})
                    VALUES({s})"""
    try:
        cur.execute(insertQ, values)
        conn.commit()
        print(f'Values Inserted: {values}')
    except (Exception, db.DatabaseError) as error:
        print(error)

In [29]:
def getData(tablename, orderby, conn):
    qu = f'select * from "{tablename}" order by "{orderby}" desc'
    alldata = pd.read_sql_query(qu, conn)
    return alldata

In [30]:
def delData(tablename, orderby, conn):
    try:
        cur = conn.cursor()
        q = f"delete from {tablename};"
        cur.execute(q)
        l = getData(tablename, orderby, conn)
        if len(l['Address'].tolist()) == 0:
            print("Delete Succesful")
    except (Exception, db.DatabaseError) as error:
        print(error)

In [31]:
def addColumns(tablename, columns, conn):
    try:
        cur = conn.cursor()
        q = f'''ALTER TABLE {tablename}
            ADD {columns};'''
        cur.execute(q)
        conn.commit()
        print(f'Columns Added')
    except (Exception, db.DatabaseError) as error:
        print(error)

In [32]:
def changeColumns(tablename, columns, conn):
    try:
        cur = conn.cursor()
        q = f'''ALTER TABLE {tablename}
            DROP COLUMN "{columns.split(' ')[0]}";'''
        cur.execute(q)
        conn.commit()
        addColumns(tablename, columns, conn)
        print(f'Columns changed')
    except (Exception, db.DatabaseError) as error:
        print(error)

In [33]:
def dropColumns(tablename, columns, conn):
    try:
        cur = conn.cursor()
        q = f'''ALTER TABLE {tablename}
            DROP COLUMN "{columns}";'''
        cur.execute(q)
        conn.commit()
        print(f'Columns dropped')
    except (Exception, db.DatabaseError) as error:
        print(error)

#### Create the Data table

In [34]:
columns1 = ""
for col in monitor2.columns.tolist():
    if columns1 == "":
        columns1 = f'"{col}" text'
    else:
        columns1 = f'{columns1}, "{col}" text'

In [35]:
columns1

'"Address" text, "Register name" text, "Scale" text, "Description" text'

In [36]:
columns2 = ""
for col in monitor2['Address'].tolist():
    if columns2 == "":
        columns2 = f'"{col}" text'
    else:
        columns2 = f'{columns2}, "{col}" text'

In [37]:
columns2

'"104" text, "207" text, "208" text, "209" text, "210" text, "211" text, "212" text, "213" text, "214" text, "215" text, "216" text, "217" text, "218" text, "219" text, "301" text, "302" text, "303" text, "304" text, "305" text, "306" text, "307" text, "308" text, "309" text, "310" text, "311" text, "312" text, "313" text, "314" text, "315" text, "320" text, "321" text, "322" text, "323" text, "324" text, "325" text, "326" text, "327" text, "328" text'

In [38]:
createTable(config.database_name,'PoC_SP_MonitoringTags',columns1,conn)

Table PoC_SP_MonitoringTags was created in DB Demo


In [39]:
createTable(config.database_name, 'PoC_SP_Metrics', columns2, conn)

Table PoC_SP_Metrics was created in DB Demo


#### Write the tag info and test if all info is available

In [41]:
column = "Address int"
changeColumns("PoC_SP_MonitoringTags", column, conn)

Columns Added
Columns changed


In [42]:
monitor2['Address'] = [int(i) for i in monitor2['Address'].tolist()]
data = monitor2.to_dict('records')
for metrics in data:
    try:
        writeValues(metrics, conn, "PoC_SP_MonitoringTags")
    except pyodbc.Error as pe:
        print("Error:", pe)
        if pe.args[0] == "08S01":  # Communication error.
            # Nuke the connection and retry.
            try:
                conn.close()
                print("Connection Ended")
            except:
                pass
            continue

Values Inserted: (104, 'PumpPowerON', 'NA', 'Power State - 0 or 1')
Values Inserted: (207, 'ActualSetpointManualHI', '0.1 ml/h', 'The actual setpoint used in operating mode "Manual".')
Values Inserted: (208, 'ActualSetpointManualLO', 'NA', 'Can be set via SetpointManual (register 00106-00107) or via the pump HMI.')
Values Inserted: (209, 'ActualPulseVolumeHI', '1 nl', 'The actual pulse volume used in operating mode "Pulse".')
Values Inserted: (210, 'ActualPulseVolumeLO', 'NA', 'Can be set via SetPulseVolume (register 00108-00109) or via the pump HMI.')
Values Inserted: (211, 'ActualBatchDosingVolumeHI', '0.001 ml', 'Can be set via SetBatchDosingVolume (register 00110-00111) or via the pump')
Values Inserted: (212, 'ActualBatchDosingVolumeLO', 'NA', 'HMI.')
Values Inserted: (213, 'ActualBatchDosingTimeHI', '0.1 s', 'The actual batch dosing time used in operating mode "Batch".')
Values Inserted: (214, 'ActualBatchDosingTimeLO', 'NA', 'Can be set via SetBatchDosingTime (register 00112-001

In [9]:
getData("PoC_SP_Metrics", "RecordID", conn)

C:\Users\ramachandran\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,103,207,208,209,210,211,212,213,214,215,...,323,324,325,326,327,328,RecordID,Timestamp,site,pumpID
0,0,62800,11,24104,1,8964,0,600,170,1,...,9100,0,8626,0,0,65535,176,2022-09-27 15:55:35.521325,DigitalHub,DH_dda001
1,0,62800,11,24104,1,8964,0,600,170,0,...,9100,0,8621,0,0,65535,175,2022-09-27 15:55:32.273924,DigitalHub,DH_dda001
2,0,62800,11,24104,1,8964,0,600,170,0,...,9100,0,8611,0,0,65535,174,2022-09-27 15:55:29.042360,DigitalHub,DH_dda001
3,0,62800,11,24104,1,8964,0,600,170,0,...,9100,0,8603,0,0,65535,173,2022-09-27 15:55:25.810953,DigitalHub,DH_dda001
4,0,62800,11,24104,1,8964,0,600,170,0,...,9100,0,8596,0,0,65535,172,2022-09-27 15:55:22.579493,DigitalHub,DH_dda001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,0,22300,11,24104,1,8964,0,600,170,0,...,8980,0,7517,0,0,65535,5,2022-09-27 14:19:20.910169,DigitalHub,DH_dda001
172,0,22300,11,24104,1,8964,0,600,170,0,...,8980,0,7514,0,0,65535,4,2022-09-27 14:19:17.694897,DigitalHub,DH_dda001
173,0,22300,11,24104,1,8964,0,600,170,0,...,8980,0,7509,0,0,65535,3,2022-09-27 14:19:11.199832,DigitalHub,DH_dda001
174,0,22300,11,24104,1,8964,0,600,170,0,...,8980,0,7505,0,0,65535,2,2022-09-27 14:19:07.952505,DigitalHub,DH_dda001


In [23]:
Logs = pd.read_csv('user_logs.csv')

In [24]:
columns3 = ""
for col in Logs.columns.tolist():
    if columns3 == "":
        columns3 = f'"{col}" text'
    else:
        columns3 = f'{columns3}, "{col}" text'

In [25]:
createTable(config.database_name, 'PoC_SP_UserLogs', columns3, conn)

Table PoC_SP_UserLogs was created in DB Demo


In [43]:
column4 = "RecordID int, Timestamp text, site text, pumpID text"
addColumns('PoC_SP_Metrics', column4, conn)

Columns Added


In [40]:
logs = getData('PoC_SP_Metrics', 'RecordID',conn)

C:\Users\ramachandran\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [41]:
logs

,103,207,208,209,210,211,212,213,214,215,...,323,324,325,326,327,328,RecordID,Timestamp,site,pumpID


In [29]:
assets = pd.read_csv('Assets.csv', delimiter='\t')

In [30]:
assets

,RecordID,Location,PumpName,PumpType
0,1,DigitalHub,DH_dda001,Grundfos DDA FCM
1,2,DigitalHub,DH_dda002,Grundfos DDA FCM
2,3,Chennai,Ch_dda003,Grundfos DDA FCM
3,4,Chicago,IL_dda004,Grundfos DDA FCM
4,5,Charlotte,NC_dda005,Grundfos DDA FCM


In [31]:
columns3 = ""
for col in assets.columns.tolist():
    if columns3 == "":
        columns3 = f'"{col}" text'
    else:
        columns3 = f'{columns3}, "{col}" text'

In [32]:
createTable(config.database_name, 'PoC_SP_Assets', columns3, conn)

Table PoC_SP_Assets was created in DB Demo


In [48]:
column4 = "Address int"
changeColumns('PoC_SP_MonitoringTags', column4, conn)

Columns Added
Columns changed


In [34]:
getData('PoC_SP_Assets', 'RecordID', conn)

C:\Users\ramachandran\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,Location,PumpName,PumpType,RecordID


In [35]:
data = assets.to_dict('records')
for metrics in data:
    try:
        writeValues(metrics, conn, "PoC_SP_Assets")
    except pyodbc.Error as pe:
        print("Error:", pe)
        if pe.args[0] == "08S01":  # Communication error.
            # Nuke the connection and retry.
            try:
                conn.close()
                print("Connection Ended")
            except:
                pass
            continue

Values Inserted: (1, 'DigitalHub', 'DH_dda001', 'Grundfos DDA FCM')
Values Inserted: (2, 'DigitalHub', 'DH_dda002', 'Grundfos DDA FCM')
Values Inserted: (3, 'Chennai', 'Ch_dda003', 'Grundfos DDA FCM')
Values Inserted: (4, 'Chicago', 'IL_dda004', 'Grundfos DDA FCM')
Values Inserted: (5, 'Charlotte', 'NC_dda005', 'Grundfos DDA FCM')


In [42]:
getData('PoC_SP_Metrics', 'RecordID',conn)

C:\Users\ramachandran\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,103,207,208,209,210,211,212,213,214,215,...,323,324,325,326,327,328,RecordID,Timestamp,site,pumpID


In [44]:
getData('PoC_SP_UserLogs', 'RecordID',conn)

C:\Users\ramachandran\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,User,Last_Access,RecordID
0,ramachandran@buckman.com,2022-09-27 22:52:46.335039,30
1,ramachandran@buckman.com,2022-09-27 15:55:20.471130,29
2,ramachandran@buckman.com,2022-09-27 15:54:44.102553,28
3,ramachandran@buckman.com,2022-09-27 15:50:47.402227,27
4,ramachandran@buckman.com,2022-09-27 15:48:32.490294,26
5,ramachandran@buckman.com,2022-09-27 15:25:01.107288,25
6,ramachandran@buckman.com,2022-09-27 15:22:53.560287,24
7,ramachandran@buckman.com,2022-09-27 15:21:05.685352,23
8,ramachandran@buckman.com,2022-09-27 15:18:33.940194,22
9,ramachandran@buckman.com,2022-09-27 15:04:11.278977,21


In [45]:
getData('PoC_SP_Assets', 'RecordID',conn)

C:\Users\ramachandran\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,Location,PumpName,PumpType,RecordID
0,Charlotte,NC_dda005,Grundfos DDA FCM,5
1,Chicago,IL_dda004,Grundfos DDA FCM,4
2,Chennai,Ch_dda003,Grundfos DDA FCM,3
3,DigitalHub,DH_dda002,Grundfos DDA FCM,2
4,DigitalHub,DH_dda001,Grundfos DDA FCM,1


In [46]:
getData('PoC_SP_MonitoringTags', 'Address',conn)

C:\Users\ramachandran\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,Register name,Scale,Description,Address
0,TimeToNextDosingLO,NA,NA,328
1,TimeToNextDosingHI,1 s,Time before the next dosing takes place (only ...,327
2,StrokeCounterLO,NA,NA,326
3,StrokeCounterHI,-,Counts the number of strokes (non-resettable).,325
4,OperatingHoursLO,NA,when the pump is dosing and when it is not dos...,324
5,OperatingHoursHI,1 s,Counts the number of hours the DDA pump has be...,323
6,RunTimeLO,NA,NA,322
7,RunTimeHI,1,Counts the time the DDA pump has been dosing (...,321
8,NumberOfPowerOns,-,Counts the number of times the pump has been p...,320
9,VolumeTripCounterLO,NA,register 00103).,315


In [47]:
df = getData("PoC_SP_Metrics", "RecordID", conn)

C:\Users\ramachandran\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [48]:
df

,104,207,208,209,210,211,212,213,214,215,...,323,324,325,326,327,328,RecordID,Timestamp,site,pumpID


In [9]:
df = df[(df['site'] == 'DigitalHub') & (df['pumpID'] == 'DH_dda001')].head(1)

In [10]:
df

,103,207,208,209,210,211,212,213,214,215,...,323,324,325,326,327,328,RecordID,Timestamp,site,pumpID
0,0,22300,11,24104,1,8964,0,600,170,0,...,9040,0,7680,0,0,65535,16,2022-09-27 14:42:54.814694,DigitalHub,DH_dda001


In [11]:
df['Timestamp'].tolist()[0]

'2022-09-27 14:42:54.814694'

In [16]:
delData("PoC_SP_Metrics", "RecordID", conn)

'Address'


C:\Users\ramachandran\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [17]:
getData("PoC_SP_Metrics", "RecordID", conn)

C:\Users\ramachandran\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,103,207,208,209,210,211,212,213,214,215,...,323,324,325,326,327,328,RecordID,Timestamp,site,pumpID


In [18]:
dropColumns("PoC_SP_Metrics", "103", conn)

Columns dropped


In [21]:
column = "'104' text"
addColumns("PoC_SP_Metrics", column, conn)

('42000', "[42000] [Microsoft][ODBC SQL Server Driver][SQL Server]Incorrect syntax near '104'. (102) (SQLExecDirectW)")
